In [442]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

pd.options.display.max_columns = 150

from requests import get

import warnings
warnings.filterwarnings('ignore')

import time
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
from playwright.async_api import expect

from bs4 import BeautifulSoup

In [9]:
df = pd.read_csv('nfl_data.csv')

In [13]:
df = df[df['PositionCategory'] == 'OFF']

In [15]:
df['Position'].unique()

array(['TE', 'OL', 'WR', nan, 'G', 'QB', 'C', 'OT', 'RB', 'FB'],
      dtype=object)

In [21]:
te = df[df['Position'] == 'TE']
wr = df[df['Position'] == 'WR']
qb = df[df['Position'] == 'QB']
bf = df[(df['Position'] == 'RB') | (df['Position'] == 'FB')]

In [22]:
te.head()

,Unnamed: 0,PlayerID,SeasonType,Season,Team,Number,Name,Position,PositionCategory,Activated,Played,Started,PassingAttempts,PassingCompletions,PassingYards,PassingCompletionPercentage,PassingYardsPerAttempt,PassingYardsPerCompletion,PassingTouchdowns,PassingInterceptions,PassingRating,PassingLong,PassingSacks,PassingSackYards,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,RushingLong,ReceivingTargets,Receptions,ReceivingYards,ReceivingYardsPerReception,ReceivingTouchdowns,ReceivingLong,Fumbles,FumblesLost,PuntReturns,PuntReturnYards,PuntReturnYardsPerAttempt,PuntReturnTouchdowns,PuntReturnLong,KickReturns,KickReturnYards,KickReturnYardsPerAttempt,KickReturnTouchdowns,KickReturnLong,SoloTackles,AssistedTackles,TacklesForLoss,Sacks,SackYards,QuarterbackHits,PassesDefended,FumblesForced,FumblesRecovered,FumbleReturnYards,FumbleReturnTouchdowns,Interceptions,InterceptionReturnYards,InterceptionReturnTouchdowns,BlockedKicks,SpecialTeamsSoloTackles,SpecialTeamsAssistedTackles,MiscSoloTackles,MiscAssistedTackles,Punts,PuntYards,PuntAverage,FieldGoalsAttempted,FieldGoalsMade,FieldGoalsLongestMade,ExtraPointsMade,TwoPointConversionPasses,TwoPointConversionRuns,TwoPointConversionReceptions,FantasyPoints,FantasyPointsPPR,ReceptionPercentage,ReceivingYardsPerTarget,Tackles,OffensiveTouchdowns,DefensiveTouchdowns,SpecialTeamsTouchdowns,Touchdowns,FantasyPosition,FieldGoalPercentage,PlayerSeasonID,FumblesOwnRecoveries,FumblesOutOfBounds,KickReturnFairCatches,PuntReturnFairCatches,PuntTouchbacks,PuntInside20,PuntNetAverage,ExtraPointsAttempted,BlockedKickReturnTouchdowns,FieldGoalReturnTouchdowns,Safeties,FieldGoalsHadBlocked,PuntsHadBlocked,ExtraPointsHadBlocked,PuntLong,BlockedKickReturnYards,FieldGoalReturnYards,PuntNetYards,SpecialTeamsFumblesForced,SpecialTeamsFumblesRecovered,MiscFumblesForced,MiscFumblesRecovered,ShortName,SafetiesAllowed,Temperature,Humidity,WindSpeed,OffensiveSnapsPlayed,DefensiveSnapsPlayed,SpecialTeamsSnapsPlayed,OffensiveTeamSnaps,DefensiveTeamSnaps,SpecialTeamsTeamSnaps,AuctionValue,AuctionValuePPR,TwoPointConversionReturns,FantasyPointsFanDuel,FieldGoalsMade0to19,FieldGoalsMade20to29,FieldGoalsMade30to39,FieldGoalsMade40to49,FieldGoalsMade50Plus,FantasyPointsDraftKings,FantasyPointsYahoo,AverageDraftPosition,AverageDraftPositionPPR,TeamID,GlobalTeamID,FantasyPointsFantasyDraft,AverageDraftPositionRookie,AverageDraftPositionDynasty,AverageDraftPosition2QB,OffensiveFumbleRecoveryTouchdowns,ScoringDetails
0,0.0,10,1,2002,LAC,81,S.Alexander,TE,OFF,14,14,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,52.5,594.7,13.2,1.4,37.3,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66.5,118.9,0,0,0.0,1.4,0.0,0.0,1.4,TE,0.0,5870629,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,S.Alexander,0.0,78,72,10,0,0,0,0,0,0,NaN,NaN,0,11.8,0.0,0.0,0.0,0.0,0.0,16.4,11.8,NaN,NaN,29,29,16.4,NaN,NaN,NaN,NaN,"[{'GameKey': '200211729', 'SeasonType': 1, 'Pl..."
2,2.0,18,1,2002,TEN,88,G.Kinne,TE,OFF,15,15,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,15.2,201.7,15.5,0.0,36.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.1,47.8,16.0,0.0,29.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.2,35.3,0,0,0.0,0.0,0.0,0.0,0.0,TE,0.0,5884471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,G.Kinne,0.0,72,72,11,0,0,0,0,0,0,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,34,34,0.0,NaN,NaN,NaN,NaN,[]
14,14.0,91,1,2002,BUF,48,B.McDonnell,TE,OFF,6,6,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,TE,0.0,586

In [23]:
wr.head()

,Unnamed: 0,PlayerID,SeasonType,Season,Team,Number,Name,Position,PositionCategory,Activated,Played,Started,PassingAttempts,PassingCompletions,PassingYards,PassingCompletionPercentage,PassingYardsPerAttempt,PassingYardsPerCompletion,PassingTouchdowns,PassingInterceptions,PassingRating,PassingLong,PassingSacks,PassingSackYards,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,RushingLong,ReceivingTargets,Receptions,ReceivingYards,ReceivingYardsPerReception,ReceivingTouchdowns,ReceivingLong,Fumbles,FumblesLost,PuntReturns,PuntReturnYards,PuntReturnYardsPerAttempt,PuntReturnTouchdowns,PuntReturnLong,KickReturns,KickReturnYards,KickReturnYardsPerAttempt,KickReturnTouchdowns,KickReturnLong,SoloTackles,AssistedTackles,TacklesForLoss,Sacks,SackYards,QuarterbackHits,PassesDefended,FumblesForced,FumblesRecovered,FumbleReturnYards,FumbleReturnTouchdowns,Interceptions,InterceptionReturnYards,InterceptionReturnTouchdowns,BlockedKicks,SpecialTeamsSoloTackles,SpecialTeamsAssistedTackles,MiscSoloTackles,MiscAssistedTackles,Punts,PuntYards,PuntAverage,FieldGoalsAttempted,FieldGoalsMade,FieldGoalsLongestMade,ExtraPointsMade,TwoPointConversionPasses,TwoPointConversionRuns,TwoPointConversionReceptions,FantasyPoints,FantasyPointsPPR,ReceptionPercentage,ReceivingYardsPerTarget,Tackles,OffensiveTouchdowns,DefensiveTouchdowns,SpecialTeamsTouchdowns,Touchdowns,FantasyPosition,FieldGoalPercentage,PlayerSeasonID,FumblesOwnRecoveries,FumblesOutOfBounds,KickReturnFairCatches,PuntReturnFairCatches,PuntTouchbacks,PuntInside20,PuntNetAverage,ExtraPointsAttempted,BlockedKickReturnTouchdowns,FieldGoalReturnTouchdowns,Safeties,FieldGoalsHadBlocked,PuntsHadBlocked,ExtraPointsHadBlocked,PuntLong,BlockedKickReturnYards,FieldGoalReturnYards,PuntNetYards,SpecialTeamsFumblesForced,SpecialTeamsFumblesRecovered,MiscFumblesForced,MiscFumblesRecovered,ShortName,SafetiesAllowed,Temperature,Humidity,WindSpeed,OffensiveSnapsPlayed,DefensiveSnapsPlayed,SpecialTeamsSnapsPlayed,OffensiveTeamSnaps,DefensiveTeamSnaps,SpecialTeamsTeamSnaps,AuctionValue,AuctionValuePPR,TwoPointConversionReturns,FantasyPointsFanDuel,FieldGoalsMade0to19,FieldGoalsMade20to29,FieldGoalsMade30to39,FieldGoalsMade40to49,FieldGoalsMade50Plus,FantasyPointsDraftKings,FantasyPointsYahoo,AverageDraftPosition,AverageDraftPositionPPR,TeamID,GlobalTeamID,FantasyPointsFantasyDraft,AverageDraftPositionRookie,AverageDraftPositionDynasty,AverageDraftPosition2QB,OffensiveFumbleRecoveryTouchdowns,ScoringDetails
4,4.0,20,1,2002,BAL,85,M.Wynn,WR,OFF,3,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,WR,0.0,5877964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,M.Wynn,0.0,58,71,8,0,0,0,0,0,0,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,3,3,0.0,NaN,NaN,NaN,NaN,[]
12,12.0,76,1,2002,NYG,87,D.Jones,WR,OFF,13,13,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,5.4,5.4,0.0,5.4,0,10.9,104.9,13.1,0.0,37.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.1,56.0,18.7,0.0,23.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,20.3,0,0,0.0,0.0,0.0,0.0,0.0,WR,0.0,5873221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,D.Jones,0.0,70,63,7,0,0,0,0,0,0,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,23,23,0.0,NaN,NaN,NaN,NaN,[]
34,34.0,212,1,2002,NYJ,87,L.Coles,WR,OFF,16,16,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.2,45.5,8.8,0.0,24.5,0,103.8,1473.8,16.6,6.8,50.1,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,189.2,293.0,0,0,0.0,6.8,0.0,0.0,6.8,WR,0.0,5878005,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,

In [24]:
qb.head()

,Unnamed: 0,PlayerID,SeasonType,Season,Team,Number,Name,Position,PositionCategory,Activated,Played,Started,PassingAttempts,PassingCompletions,PassingYards,PassingCompletionPercentage,PassingYardsPerAttempt,PassingYardsPerCompletion,PassingTouchdowns,PassingInterceptions,PassingRating,PassingLong,PassingSacks,PassingSackYards,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,RushingLong,ReceivingTargets,Receptions,ReceivingYards,ReceivingYardsPerReception,ReceivingTouchdowns,ReceivingLong,Fumbles,FumblesLost,PuntReturns,PuntReturnYards,PuntReturnYardsPerAttempt,PuntReturnTouchdowns,PuntReturnLong,KickReturns,KickReturnYards,KickReturnYardsPerAttempt,KickReturnTouchdowns,KickReturnLong,SoloTackles,AssistedTackles,TacklesForLoss,Sacks,SackYards,QuarterbackHits,PassesDefended,FumblesForced,FumblesRecovered,FumbleReturnYards,FumbleReturnTouchdowns,Interceptions,InterceptionReturnYards,InterceptionReturnTouchdowns,BlockedKicks,SpecialTeamsSoloTackles,SpecialTeamsAssistedTackles,MiscSoloTackles,MiscAssistedTackles,Punts,PuntYards,PuntAverage,FieldGoalsAttempted,FieldGoalsMade,FieldGoalsLongestMade,ExtraPointsMade,TwoPointConversionPasses,TwoPointConversionRuns,TwoPointConversionReceptions,FantasyPoints,FantasyPointsPPR,ReceptionPercentage,ReceivingYardsPerTarget,Tackles,OffensiveTouchdowns,DefensiveTouchdowns,SpecialTeamsTouchdowns,Touchdowns,FantasyPosition,FieldGoalPercentage,PlayerSeasonID,FumblesOwnRecoveries,FumblesOutOfBounds,KickReturnFairCatches,PuntReturnFairCatches,PuntTouchbacks,PuntInside20,PuntNetAverage,ExtraPointsAttempted,BlockedKickReturnTouchdowns,FieldGoalReturnTouchdowns,Safeties,FieldGoalsHadBlocked,PuntsHadBlocked,ExtraPointsHadBlocked,PuntLong,BlockedKickReturnYards,FieldGoalReturnYards,PuntNetYards,SpecialTeamsFumblesForced,SpecialTeamsFumblesRecovered,MiscFumblesForced,MiscFumblesRecovered,ShortName,SafetiesAllowed,Temperature,Humidity,WindSpeed,OffensiveSnapsPlayed,DefensiveSnapsPlayed,SpecialTeamsSnapsPlayed,OffensiveTeamSnaps,DefensiveTeamSnaps,SpecialTeamsTeamSnaps,AuctionValue,AuctionValuePPR,TwoPointConversionReturns,FantasyPointsFanDuel,FieldGoalsMade0to19,FieldGoalsMade20to29,FieldGoalsMade30to39,FieldGoalsMade40to49,FieldGoalsMade50Plus,FantasyPointsDraftKings,FantasyPointsYahoo,AverageDraftPosition,AverageDraftPositionPPR,TeamID,GlobalTeamID,FantasyPointsFantasyDraft,AverageDraftPositionRookie,AverageDraftPositionDynasty,AverageDraftPosition2QB,OffensiveFumbleRecoveryTouchdowns,ScoringDetails
11,11.0,65,1,2002,IND,7,B.Huard,QB,OFF,16,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,QB,0.0,5874620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B.Huard,0.0,72,63,7,0,0,0,0,0,0,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,14,14,0.0,NaN,NaN,NaN,NaN,[]
31,31.0,198,1,2002,PHI,10,K.Detmer,QB,OFF,14,14,1,32.6,22.2,261.2,79.2,10.9,13.8,2.7,0.0,135.0,43.1,1.4,10.9,2.7,5.4,2.7,1.4,4.1,0,0.0,0.0,0.0,0.0,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.2,27.2,0,0,0.0,1.4,0.0,0.0,1.4,QB,0.0,5864954,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,K.Detmer,0.0,68,68,12,0,0,0,0,0,0,NaN,NaN,0,0.2,0.0,0.0,0.0,0.0,0.0,0.2,0.2,NaN,NaN,26,26,0.2,NaN,NaN,NaN,NaN,"[{'GameKey': '200211231', 'SeasonType': 1, 'Pl..."
33,33.0,211,1,2002,SF,11,B.Doman,QB,OFF,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,QB,0.0,5875277,0.0,0.0,0.0,0

In [25]:
bf.head()

,Unnamed: 0,PlayerID,SeasonType,Season,Team,Number,Name,Position,PositionCategory,Activated,Played,Started,PassingAttempts,PassingCompletions,PassingYards,PassingCompletionPercentage,PassingYardsPerAttempt,PassingYardsPerCompletion,PassingTouchdowns,PassingInterceptions,PassingRating,PassingLong,PassingSacks,PassingSackYards,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,RushingLong,ReceivingTargets,Receptions,ReceivingYards,ReceivingYardsPerReception,ReceivingTouchdowns,ReceivingLong,Fumbles,FumblesLost,PuntReturns,PuntReturnYards,PuntReturnYardsPerAttempt,PuntReturnTouchdowns,PuntReturnLong,KickReturns,KickReturnYards,KickReturnYardsPerAttempt,KickReturnTouchdowns,KickReturnLong,SoloTackles,AssistedTackles,TacklesForLoss,Sacks,SackYards,QuarterbackHits,PassesDefended,FumblesForced,FumblesRecovered,FumbleReturnYards,FumbleReturnTouchdowns,Interceptions,InterceptionReturnYards,InterceptionReturnTouchdowns,BlockedKicks,SpecialTeamsSoloTackles,SpecialTeamsAssistedTackles,MiscSoloTackles,MiscAssistedTackles,Punts,PuntYards,PuntAverage,FieldGoalsAttempted,FieldGoalsMade,FieldGoalsLongestMade,ExtraPointsMade,TwoPointConversionPasses,TwoPointConversionRuns,TwoPointConversionReceptions,FantasyPoints,FantasyPointsPPR,ReceptionPercentage,ReceivingYardsPerTarget,Tackles,OffensiveTouchdowns,DefensiveTouchdowns,SpecialTeamsTouchdowns,Touchdowns,FantasyPosition,FieldGoalPercentage,PlayerSeasonID,FumblesOwnRecoveries,FumblesOutOfBounds,KickReturnFairCatches,PuntReturnFairCatches,PuntTouchbacks,PuntInside20,PuntNetAverage,ExtraPointsAttempted,BlockedKickReturnTouchdowns,FieldGoalReturnTouchdowns,Safeties,FieldGoalsHadBlocked,PuntsHadBlocked,ExtraPointsHadBlocked,PuntLong,BlockedKickReturnYards,FieldGoalReturnYards,PuntNetYards,SpecialTeamsFumblesForced,SpecialTeamsFumblesRecovered,MiscFumblesForced,MiscFumblesRecovered,ShortName,SafetiesAllowed,Temperature,Humidity,WindSpeed,OffensiveSnapsPlayed,DefensiveSnapsPlayed,SpecialTeamsSnapsPlayed,OffensiveTeamSnaps,DefensiveTeamSnaps,SpecialTeamsTeamSnaps,AuctionValue,AuctionValuePPR,TwoPointConversionReturns,FantasyPointsFanDuel,FieldGoalsMade0to19,FieldGoalsMade20to29,FieldGoalsMade30to39,FieldGoalsMade40to49,FieldGoalsMade50Plus,FantasyPointsDraftKings,FantasyPointsYahoo,AverageDraftPosition,AverageDraftPositionPPR,TeamID,GlobalTeamID,FantasyPointsFantasyDraft,AverageDraftPositionRookie,AverageDraftPositionDynasty,AverageDraftPosition2QB,OffensiveFumbleRecoveryTouchdowns,ScoringDetails
22,22.0,138,1,2002,NYG,29,D.Washington,RB,OFF,12,12,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.4,89.8,22.4,0.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.2,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,9.5,0.0,0.0,0.0,0.0,RB,0.0,5864554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,D.Washington,0.0,71,63,8,0,0,0,0,0,0,NaN,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,23,23,0.0,NaN,NaN,NaN,NaN,[]
27,27.0,170,1,2002,BAL,31,J.Lewis,RB,OFF,16,16,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,359.1,1547.3,5.8,8.2,87.4,0,54.8,515.4,11.0,1.4,89.8,10.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,255.2,310.0,0,0,0.0,9.5,0.0,0.0,9.5,RB,0.0,5880987,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,J.Lewis,0.0,70,62,10,0,0,0,0,0,0,NaN,NaN,0,151.2,0.0,0.0,0.0,0.0,0.0,178.6,151.2,NaN,NaN,3,3,178.6,NaN,NaN,NaN,NaN,"[{'GameKey': '200210403', 'SeasonType': 1, 'Pl..."
36,36.0,224,1,2002,TEN,20,M.Green,RB,OFF,15,15,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.5,82.8,4.6,0.0,14.0,0,9.5,66.5,11.0,1.4,24.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,11.7,11.7,0.0,11.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.9,30.1,0,0,1.4,1.4,0.

In [222]:
url = 'https://football.fantasysports.yahoo.com/f1/14104/matchup?week=11&mid1=1'

In [535]:
async def get_html(url, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                await page.mouse.wheel(0,15000)
                
                await page.locator('body:has-text("Final results")').click()
                html = await page.content()
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            html = '0'
            continue
        else:
            break
    return html

In [509]:
html = await get_html(url)

In [775]:
def clean_html(html, league_id, week, num_players, formt):
    
    scores = []
    team_num = []
    team_name = []
    opp_score = []
    win_loss = []

    score_list = re.findall(r'>\d+\.\d+<', html)[-num_players:]
    for score in score_list:
        scores.append(score[1:-1])
        
    names = re.findall(fr'{league_id}/\d+\">[^<]+<',html)[2:]
    for name in names:
        num = re.findall(r'/\d+', name)
        team_num.append(num[0][1:])
        name = re.findall(r'>.+<', name)
        team_name.append(name[0][1:-1])
        
    if week % 5 == 0:
        print(league_id)
        print(week)
        print(team_num)
        print(team_name)
        print(scores)

    league_data = pd.DataFrame({'league':league_id , 'format': formt, 'week':week,'team number':team_num,
                                'team name':team_name,'score':scores})
    
    league_data['week'] = league_data['week'].astype(int)
    league_data['team number'] = league_data['team number'].astype(int)
    league_data['score'] = league_data['score'].astype(float)
    
    for i in range(len(league_data)):
        if i % 2 == 0:
            opp_score.append(league_data.loc[(i+1), 'score'])
        else:
            opp_score.append(league_data.loc[(i-1), 'score'])
        
        if league_data.loc[i, 'score'] > opp_score[i]:
            win_loss.append(1)
        else:
            win_loss.append(0)
            
    league_data['opponent score'] = opp_score
    league_data['W/L'] = win_loss
    
    league_data['opponent score'] = league_data['opponent score'].astype(float)

            
    return league_data

In [776]:
fantasy_data = pd.DataFrame(columns=['league','format','week','team number','team name','score'])

In [771]:
url = 'https://football.fantasysports.yahoo.com/f1/14104/matchup?week=11&mid1=1'

In [777]:
async def get_data(fantasy_data, league_id, num_weeks, num_players, formt):
    
    for week in range(1,num_weeks+1):
        url = f'https://football.fantasysports.yahoo.com/f1/{league_id}/matchup?week={week}&mid1=1'
        
        html = await get_html(url)
        
        if html != '0':
            week_data = clean_html(html, league_id, week, num_players, formt)

            fantasy_data = pd.concat([fantasy_data, week_data])
            
            if week % 10 == 0: 
                print(f'Week {week} completed')
                
        else:
            break
    return fantasy_data

In [778]:
fantasy_data = await get_data(fantasy_data, '14104', 15, 10, .5)

14104
5
['1', '6', '2', '5', '3', '4', '7', '9', '8', '10']
["Bryan's Boss Team", "joe's Unbelievable Team", "Freddie's Phantoms", 'Colliers Wood Crows', 'Big Slum', "James's Marvelous Team", "Bobby's Boss Team", 'SPERMINATING DOOM', 'Costly Errors', 'Skidmarx']
['150.74', '87.32', '136.56', '104.66', '123.18', '100.26', '136.36', '67.56', '95.26', '99.58']
14104
10
['1', '2', '3', '9', '4', '8', '5', '7', '6', '10']
["Bryan's Boss Team", "Freddie's Phantoms", 'Big Slum', 'SPERMINATING DOOM', "James's Marvelous Team", 'Costly Errors', 'Colliers Wood Crows', "Bobby's Boss Team", "joe's Unbelievable Team", 'Skidmarx']
['99.46', '103.60', '110.48', '107.14', '74.38', '98.64', '102.50', '122.92', '81.68', '106.44']
Week 10 completed
14104
15
['1', '7', '2', '6', '3', '5', '4', '10', '8', '9']
["Bryan's Boss Team", "Bobby's Boss Team", "Freddie's Phantoms", "joe's Unbelievable Team", 'Big Slum', 'Colliers Wood Crows', "James's Marvelous Team", 'Skidmarx', 'Costly Errors', 'SPERMINATING DOOM

In [779]:
fantasy_data

,league,format,week,team number,team name,score,opponent score,W/L
0,14104,0.5,1,1,Bryan's Boss Team,69.58,125.18,0.0
1,14104,0.5,1,2,Freddie's Phantoms,125.18,69.58,1.0
2,14104,0.5,1,3,Big Slum,90.32,77.82,1.0
3,14104,0.5,1,9,SPERMINATING DOOM,77.82,90.32,0.0
4,14104,0.5,1,4,James's Marvelous Team,133.42,132.20,1.0
...,...,...,...,...,...,...,...,...
5,14104,0.5,15,5,Colliers Wood Crows,131.80,98.20,1.0
6,14104,0.5,15,4,James's Marvelous Team,67.88,110.94,0.0
7,14104,0.5,15,10,Skidmarx,110.94,67.88,1.0
8,14104,0.5,15,8,Costly Errors,102.70,113.48,0.0


In [400]:
fantasy_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 9
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   league          150 non-null    object 
 1   week            150 non-null    object 
 2   team number     150 non-null    object 
 3   team name       150 non-null    object 
 4   score           150 non-null    float64
 5   opponent score  150 non-null    float64
 6   W/L             150 non-null    float64
dtypes: float64(3), object(4)
memory usage: 9.4+ KB


In [399]:
fantasy_data.to_csv('fantasy_data.csv')

In [401]:
fantasy_data[fantasy_data['score'] < 70]['W/L'].mean()

0.125

In [402]:
fantasy_data[fantasy_data['W/L']==1]['score'].mean()

118.89999999999998

In [403]:
fantasy_data['score'].mean()

107.1684

In [404]:
fantasy_data['opponent score'].mean()

107.16840000000002

In [407]:
fantasy_data.groupby('team name')['score'].mean()

team name
Big Slum                   100.324000
Bobby's Boss Team          120.701333
Bryan's Boss Team          100.112000
Colliers Wood Crows        105.092000
Costly Errors               97.521333
Freddie's Phantoms         117.650667
James's Marvelous Team      95.741333
SPERMINATING DOOM          107.841333
Skidmarx                   115.218667
joe's Unbelievable Team    111.481333
Name: score, dtype: float64

In [533]:
league_ids = ['1000004', '1000007', '1000014', '1000018']

In [539]:
for league in league_ids:
    fantasy_data = await get_data(fantasy_data, league, 15, 10)

Timeout error on https://football.fantasysports.yahoo.com/f1/1000004/matchup?week=1&mid1=1
Timeout error on https://football.fantasysports.yahoo.com/f1/1000004/matchup?week=1&mid1=1
Timeout error on https://football.fantasysports.yahoo.com/f1/1000004/matchup?week=1&mid1=1
1000007
1
['1', '2', '3', '9', '4', '8', '5', '7', '6', '10']
["Jesse's Remarkable Team", "Brittany's Neat Team", 'Terrible Team', "the's World-Class Team", "derrek's Quality Team", "Dante's Dandy Team", "Smallow's Standout Team", "Aiden's Amazing Team", 'Julie', "catalin's Great Team"]
['110.88', '84.80', '93.10', '120.82', '149.58', '86.68', '106.76', '111.06', '141.42', '125.50']
1000007
2
['1', '3', '2', '10', '4', '9', '5', '8', '6', '7']
["Jesse's Remarkable Team", 'Terrible Team', "Brittany's Neat Team", "catalin's Great Team", "derrek's Quality Team", "the's World-Class Team", "Smallow's Standout Team", "Dante's Dandy Team", 'Julie', "Aiden's Amazing Team"]
['98.94', '113.36', '92.30', '120.78', '130.18', '132

Error: NS_ERROR_UNKNOWN_HOST
=========================== logs ===========================
navigating to "https://football.fantasysports.yahoo.com/f1/1000014/matchup?week=1&mid1=1", waiting until "load"
============================================================

In [540]:
import pandas as pd
import time

def get_yahoo_leagues(beg,num):    
    '''
    Scrape yahoo leagues to determine league id, points format, and teams in league
    Returns a df
    '''
    league_id = []
    league_player_count = []
    league_scoring_format = []
    league_positions = []
    
    for i in range (beg,num+1):
        try:
            df = pd.read_html(f'https://football.fantasysports.yahoo.com/f1/{i}/settings') 
            df0 = df[0]
            df1 = df[1]
            league_id.append(i)
            league_scoring_format.append(df1[df1.iloc[:,0].str.contains('Receptions')].iloc[0,1])
            league_player_count.append(int(df0[df0.Setting=='Max Teams:']['Value'].iloc[0]))
            league_positions.append(df[1].iloc[:,0].name)
          
        except:
            pass

        time.sleep(5)

    return pd.DataFrame({'id':league_id,
                         'player_count':league_player_count,'format':league_scoring_format,
                         'positions':league_positions})

In [586]:
df = get_yahoo_leagues(100700,100750)
df.to_csv('league_ids.csv')

In [587]:
df

,id,player_count,format,positions


In [552]:
df1 = df1.append(df, ignore_index=True)

In [553]:
df1

,id,player_count,format,positions
0,100025,12,1,"(Offense, Defense/Special Teams)"
1,100026,10,1,"(Offense, Kickers, Defense/Special Teams)"
2,100031,12,.1,"(Offense, Kickers, Defense/Special Teams)"


In [593]:
import json

In [607]:
f = open('ffl.json')

In [608]:
data = json.load(f)

In [654]:
data

{'399.l.343688': {'420 League': {'Week_1': {'matchup1': {'BMS Goats': '145.40',
     'The Saddlebackers': '130.28'},
    'matchup2': {'Holy bong water': '167.64', 'Judge Jeudy': '147.42'},
    'matchup3': {'Big dick bandits': '113.06', 'Hollywood City': '122.10'},
    'matchup4': {'Watson your wallet': '164.00', 'My Meat': '117.98'},
    'matchup5': {'Highlted Chillie': '167.80', 'I got a Thielen': '139.12'}},
   'Week_2': {'matchup1': {'BMS Goats': '170.38',
     'Big dick bandits': '105.28'},
    'matchup2': {'Holy bong water': '111.50', 'The Saddlebackers': '171.40'},
    'matchup3': {'Watson your wallet': '183.80', 'Hollywood City': '128.54'},
    'matchup4': {'Highlted Chillie': '109.02', 'My Meat': '98.02'},
    'matchup5': {'I got a Thielen': '107.32', 'Judge Jeudy': '134.40'}},
   'Week_3': {'matchup1': {'BMS Goats': '153.76',
     'Watson your wallet': '88.10'},
    'matchup2': {'Holy bong water': '156.22', 'Big dick bandits': '133.12'},
    'matchup3': {'The Saddlebackers': '

In [725]:
df = pd.DataFrame()

In [728]:
team_name = []
scores = []
week = []
team_num = []
num = 0

df = pd.DataFrame(columns=['league_id','week','team number','team name','score'])

for i in range(1,16):
    team_name = []
    scores = []
    week = []
    team_num = []
    num = 0
    for key in data['399.l.343688']['420 League'][f'Week_{i}']:
        for name in data['399.l.343688']['420 League'][f'Week_{i}'][key]:
            num += 1
            team_num.append(num)
            week.append(i)
            team_name.append(name)
            scores.append(data['399.l.343688']['420 League'][f'Week_{i}'][key][name])
    
    df1 = pd.DataFrame({'league_id':'399.l.343688','week':i,'team number':team_num,
                  'team name':team_name,'score':scores})
    df = df.append(df1)

In [736]:
df=df.reset_index(drop=True)

In [737]:
df

,league_id,week,team number,team name,score
0,399.l.343688,1,1,BMS Goats,145.40
1,399.l.343688,1,2,The Saddlebackers,130.28
2,399.l.343688,1,3,Holy bong water,167.64
3,399.l.343688,1,4,Judge Jeudy,147.42
4,399.l.343688,1,5,Big dick bandits,113.06
...,...,...,...,...,...
137,399.l.343688,15,4,The Saddlebackers,109.54
138,399.l.343688,15,5,Big dick bandits,154.52
139,399.l.343688,15,6,I got a Thielen,98.84
140,399.l.343688,15,7,My Meat,134.06


In [738]:
for i in range(1,16):
    team_name = []
    scores = []
    week = []
    team_num = []
    num = 0
    for key in data['406.l.147449']['Madison\'s Finest'][f'Week_{i}']:
        for name in data['406.l.147449']['Madison\'s Finest'][f'Week_{i}'][key]:
            num += 1
            team_num.append(num)
            week.append(i)
            team_name.append(name)
            scores.append(data['406.l.147449']['Madison\'s Finest'][f'Week_{i}'][key][name])
    
    df1 = pd.DataFrame({'league_id':'406.l.147449','week':i,'team number':team_num,
                  'team name':team_name,'score':scores})
    df = df.append(df1)

In [739]:
df

,league_id,week,team number,team name,score
0,399.l.343688,1,1,BMS Goats,145.40
1,399.l.343688,1,2,The Saddlebackers,130.28
2,399.l.343688,1,3,Holy bong water,167.64
3,399.l.343688,1,4,Judge Jeudy,147.42
4,399.l.343688,1,5,Big dick bandits,113.06
...,...,...,...,...,...
7,406.l.147449,15,8,TLSmooth,93.90
8,406.l.147449,15,9,*,63.60
9,406.l.147449,15,10,Team Jake,97.68
10,406.l.147449,15,11,FIRST2LAST,110.42


In [740]:
for i in range(1,16):
    team_name = []
    scores = []
    week = []
    team_num = []
    num = 0
    for key in data['406.l.147473']['420 League'][f'Week_{i}']:
        for name in data['406.l.147473']['420 League'][f'Week_{i}'][key]:
            num += 1
            team_num.append(num)
            week.append(i)
            team_name.append(name)
            scores.append(data['406.l.147473']['420 League'][f'Week_{i}'][key][name])
    
    df1 = pd.DataFrame({'league_id':'406.l.147473','week':i,'team number':team_num,
                  'team name':team_name,'score':scores})
    df = df.append(df1)

In [741]:
df

,league_id,week,team number,team name,score
0,399.l.343688,1,1,BMS Goats,145.40
1,399.l.343688,1,2,The Saddlebackers,130.28
2,399.l.343688,1,3,Holy bong water,167.64
3,399.l.343688,1,4,Judge Jeudy,147.42
4,399.l.343688,1,5,Big dick bandits,113.06
...,...,...,...,...,...
7,406.l.147473,15,8,Big dick bandits,147.40
8,406.l.147473,15,9,The Saddlebackers,121.14
9,406.l.147473,15,10,No Clever Name,107.36
10,406.l.147473,15,11,Team Supreme,133.22


In [742]:
for i in range(1,16):
    team_name = []
    scores = []
    week = []
    team_num = []
    num = 0
    for key in data['414.l.561988']['420 League'][f'Week_{i}']:
        for name in data['414.l.561988']['420 League'][f'Week_{i}'][key]:
            num += 1
            team_num.append(num)
            week.append(i)
            team_name.append(name)
            scores.append(data['414.l.561988']['420 League'][f'Week_{i}'][key][name])
    
    df1 = pd.DataFrame({'league_id':'414.l.561988','week':i,'team number':team_num,
                  'team name':team_name,'score':scores})
    df = df.append(df1)

In [756]:
df = df.reset_index(drop=True)

In [757]:
df['score'] = df['score'].astype(float)

In [758]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 670 entries, 0 to 669
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   league_id    670 non-null    object 
 1   week         670 non-null    object 
 2   team number  670 non-null    object 
 3   team name    670 non-null    object 
 4   score        670 non-null    float64
dtypes: float64(1), object(4)
memory usage: 26.3+ KB


In [760]:
opp_score = []
win_loss = []
for i in range(len(df)):
    if i % 2 == 0:
        opp_score.append(df.loc[(i+1), 'score'])
    else:
        opp_score.append(df.loc[(i-1), 'score'])

    if df.loc[i, 'score'] > opp_score[i]:
        win_loss.append(1)
    else:
        win_loss.append(0)
            
df['opponent score'] = opp_score
df['W/L'] = win_loss

df['opponent score'] = df['opponent score'].astype(float)

In [780]:
df

,league_id,week,team number,team name,score,opponent score,W/L
0,399.l.343688,1,1,BMS Goats,145.40,130.28,1
1,399.l.343688,1,2,The Saddlebackers,130.28,145.40,0
2,399.l.343688,1,3,Holy bong water,167.64,147.42,1
3,399.l.343688,1,4,Judge Jeudy,147.42,167.64,0
4,399.l.343688,1,5,Big dick bandits,113.06,122.10,0
...,...,...,...,...,...,...,...
665,414.l.561988,15,8,Big dick bandits,130.20,104.46,1
666,414.l.561988,15,9,The Saddlebackers,80.98,117.00,0
667,414.l.561988,15,10,Team Supreme,117.00,80.98,1
668,414.l.561988,15,11,Ligma Nutz,106.00,121.70,0
